In [1]:
import pandas as pd
from ssri_interactions.config import Config
from ssri_interactions.interactions.popcup import PopulationCoupling
from ssri_interactions.interactions.preprocessors import InteractionsPreprocessor
from ssri_interactions.interactions.loaders import SpontaneousActivityLoader
from ssri_interactions.interactions.pairwise import PairwiseCorr, PairwisePartialCorr
from ssri_interactions.interactions.runners import InteractionsRunner
from ssri_interactions.io import load_derived_generic
from ssri_interactions.config import Config
import numpy as np

dd = Config.derived_data_dir / "corrs"
dd.mkdir(exist_ok=True)

c:\ProgramData\Miniconda3\envs\ssri\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
from ssri_interactions.transforms.graph import GraphTransformer
from ssri_interactions.io import load_distances


def add_meta_pairwise(df_edge, comb_col="comb", value_coll="corr"):
    neuron_types = load_derived_generic("neuron_types.csv")
    df_rsc = df_edge.rename(columns={comb_col: "comb_id"}).copy()
    df_distance = load_distances().rename(
        columns=dict(neuron1="neuron_1", neuron2="neuron_2")
    )
    g_transformer = GraphTransformer(
        relabel_nodes=True,
        weight_attr=value_coll,
        neuron_type_col="neuron_type",
        distance_source_col="neuron_1",
        distance_target_col="neuron_2",
        df_distance=df_distance,
    )
    df_distance = g_transformer._add_id_comb_col(
        df_distance,
        source_col="neuron_1",
        target_col="neuron_2",
        comb_col_name="comb_id",
    )
    df_rsc = g_transformer._add_id_comb_col(
        df_rsc,
        source_col="neuron_1",
        target_col="neuron_2",
        comb_col_name="comb_id",
    )
    df_rsc = g_transformer._add_neuron_type_comb_col(
        df_rsc,
        df_neuron_types=neuron_types,
        source_col="neuron_1",
        target_col="neuron_2",
    )
    thresh = (
        df_rsc.query("shuffle==True")
        .groupby(["nt_comb", "bin_width"])[value_coll]
        .apply(lambda x: np.quantile(a=x.dropna().abs(), q=0.99))
        .to_dict()
    )
    df_rsc = (
        df_rsc.groupby(["nt_comb", "bin_width"])
        .apply(lambda x: thresh[x.name])
        .to_frame("thresh")
        .reset_index()
        .merge(df_rsc)
        .assign(sig=lambda x: x[value_coll].abs() > x["thresh"])
    )
    return df_rsc.merge(df_distance[["comb_id", "distance"]]).merge(
        neuron_types[["session_name", "group"]].drop_duplicates()
    )


def add_meta_single(df_node):
    neuron_types = load_derived_generic("neuron_types.csv")
    df_node = df_node.merge(
        neuron_types[
            ["neuron_id", "session_name", "neuron_type", "group"]
        ].drop_duplicates()
    )
    thresh = (
        df_node.query("shuffle==True")
        .groupby(["neuron_type", "bin_width"])["cc"]
        .apply(lambda x: np.quantile(a=x.dropna().abs(), q=0.95))
        .to_dict()
    )
    df_node = (
        df_node.groupby(["neuron_type", "bin_width"])
        .apply(lambda x: thresh[x.name])
        .to_frame("thresh")
        .reset_index()
        .merge(df_node)
        .assign(sig=lambda x: x["cc"].abs() > x["thresh"])
    )
    return df_node


## Get Pairwise Interactions

In [6]:
neuron_types = load_derived_generic("neuron_types.csv")
sessions = neuron_types["session_name"].dropna().unique()

res_corr = []
res_pcorr = []
res_pcup = []
bin_widths = [0.05, 0.1, 0.5, 1]

loader_fac = lambda bin_width, shuffle: SpontaneousActivityLoader(
    session_name=sessions[0], bin_width=bin_width, shuffle=shuffle
)
preprocessor_fac = lambda: InteractionsPreprocessor(
    z=True, minmax=False, gaussian_sigma=None
)
corr_fac = lambda: PairwiseCorr()
pcorr_fac = lambda: PairwisePartialCorr()
pcup_fac = lambda: PopulationCoupling()

runner_fac = lambda bin_width, shuffle: InteractionsRunner(
    loader=loader_fac(bin_width, shuffle=shuffle),
    preprocessor=preprocessor_fac(),
    pcup=pcup_fac(),
    corr=corr_fac(),
    pcorr=pcorr_fac(),
)
for shuffle in (True, False):
    for bin_width in bin_widths:
        runner = runner_fac(bin_width, shuffle)
        res_pcorr.append(runner.run_pcorr_multi(sessions).assign(bin_width=bin_width, shuffle=shuffle))
        res_corr.append(runner.run_corr_multi(sessions).assign(bin_width=bin_width, shuffle=shuffle))
        res_pcup.append(runner.run_pcup_multi(sessions).assign(bin_width=bin_width, shuffle=shuffle))


df_corr = pd.concat(res_corr).reset_index(drop=True).pipe(add_meta_pairwise, value_coll="corr")
df_pcorr = pd.concat(res_pcorr).reset_index(drop=True).pipe(add_meta_pairwise, value_coll="pcorr")
df_pcup = pd.concat(res_pcup).reset_index(drop=True).pipe(add_meta_single)

df_corr.to_parquet(dd / "spont - corr.parquet", index=False)
df_pcorr.to_parquet(dd / "spont - pcorr.parquet", index=False)
df_pcup.to_parquet(dd / "spont - pcup.parquet", index=False)